<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/lesson11/NLP_intro_hw_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Взять предобученную трансформерную архитектуру и решить задачу перевода (для того же корпуса что вы выбрали из предыдущего дз)

In [36]:
!pip install transformers -q
!pip install transformers[sentencepiece] -q
import transformers
from transformers import AutoTokenizer
import io
import re
import random

In [37]:
!wget http://www.manythings.org/anki/rus-eng.zip

!rm -rf rus-eng
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng

--2023-03-29 07:50:42--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15374406 (15M) [application/zip]
Saving to: ‘rus-eng.zip.5’

rus-eng.zip.5       100%[===================>]  14.66M  7.33MB/s    in 2.0s    

2023-03-29 07:50:44 (7.33 MB/s) - ‘rus-eng.zip.5’ saved [15374406/15374406]

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [38]:
path_to_file = "rus-eng/rus.txt"

In [39]:
# def preprocess_sentence(w):
#   w = w.lower().strip()

#   # creating a space between a word and the punctuation following it
#   # eg: "he is a boy." => "he is a boy ."
#   # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
#   w = re.sub(r"([?.!,])", r" \1 ", w)
#   w = re.sub(r'[" "]+', " ", w)

#   # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
#   w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

#   w = w.strip()

#   # adding a start and an end token to the sentence
#   # so that the model know when to start and stop predicting.
#   w = ' ' + w + ' '
#   return w    

In [40]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  # word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]
  word_pairs = [[w for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [41]:
en, ru = create_dataset(path_to_file, None)
print(en[30])
print(ru[30])

Hide.
Спрячьтесь.


In [46]:
!pip install sacremoses -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [47]:
# Загрузка предобученного токенайзера
# Загрузим предобученную модель для перевода с русского на английский с HuggingFace.

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BertTokenizer

# tokenizer = BertTokenizer.from_pretrained("bert-base-cased", do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en", do_lower_case=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

In [48]:
# Пример перевода
inputs = tokenizer.encode('Я иду по улице.', return_tensors='pt')
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]).replace('<pad> ', ''))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


I'm walking down the street.</s>


In [49]:
ids = random.choices(range(len(ru)), k=25)

input_texts = []
target_texts = []
prediction_texts = []

for id in ids:
    input_texts.append(ru[id])
    target_texts.append(en[id])
    
    inputs = tokenizer.encode(ru[id], return_tensors='pt')
    outputs = model.generate(inputs)
    prediction_texts.append(tokenizer.decode(outputs[0]).replace('<pad> ', ''))

Отобрали случайным образом 25 предложений из текста. и перевели.

In [50]:
import pandas as pd
df = pd.DataFrame(list(zip(input_texts, prediction_texts, target_texts)), 
                  columns=['Input_text', 'Prediction_text', 'Target_text'])
df

,Input_text,Prediction_text,Target_text
0,Выбора нет.,There's no choice.</s>,There is no choice.
1,У нас ограниченный бюджет.,We have a limited budget.</s>,We're on a tight budget.
2,Мы отложили это событие.,We've postponed the event.</s>,We postponed the event.
3,Я могу это уладить.,I can handle this.</s>,I can handle that.
4,Тебе спать не пора?,Shouldn't you be sleeping?</s>,Isn't it about time you went to bed?
5,Я боюсь молнии.,I'm afraid of lightning.</s>,I'm scared of lightning.
6,Давай поиграем в бейсбол после занятий.,Let's play baseball after class.</s>,Let's play baseball after school.
7,"Том хочет, чтобы всё оставалось как есть.",Tom wants things to stay the way they are.</s>,Tom wants to keep things as they are.
8,Я профессор.,I'm a professor.</s>,I am a professor.
9,"Неважно, что я думаю.",It doesn't matter what I think.</s>,It doesn't matter what I think.


Качество перевода значительно улучшилось по сравнению с RNN с вниманием. Имена  не были переведены.